# Digit Recognition Model

In [223]:
# Setting a few hyper parameters
NUM_CLASSES = 10
NUM_FEATURES = 900
RANDOM_SEED = 42

### Loading the data
The data files that we have are :
1. X_data.pt:
<br>
    This file has the slate matrix data.

2. y_data.pt:
<br>
    This file has the y labels for all the slates.


In [224]:
# Importing the necessities 
import torch

X_data = torch.load('../X_data.pt').numpy()
y_data = torch.load('../y_data.pt').numpy()

X_data[:5],y_data[:5]

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([1, 1, 1, 1, 1]))

### Creating the train and test split

train and test split ration is 80-20.

In [225]:
# Using the sklearn
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_data,
    y_data,
    test_size=0.2,
    random_state=RANDOM_SEED
)

X_train[:5],y_train[:5]

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([3, 0, 7, 5, 6]))

### Creating the Classification Model
Our model taken in 900 input points as a matrix, and classifies the matrix into 10 digits.

* <code> in_features </code> : 900
* <code> out_features </code> : 10

We will have 2 hidden layers with 16 features each.

In [226]:
# Import nn
from torch import nn

class DigitModel(nn.Module):
    """This is the multiclass classification model for matrix to digits."""
    
    def __init__(self, in_features = 900):
        """Create an instance of the model."""
        super().__init__()
        self.linear_stack = nn.Sequential(
            nn.Linear(
                in_features=in_features,
                out_features=16
            ),
            nn.Sigmoid(),
            nn.Linear(
                in_features=16,
                out_features=16
            ),
            nn.Sigmoid(),
            nn.Linear(
                in_features=16,
                out_features=10
            )     
        )
        
    def forward(self,x):
        return self.linear_stack(x)
    
    def logits_to_preds(self,logits):
        """Convert Logits to predictions."""
        probs = torch.softmax(logits,
                              dim=1)
        preds = torch.argmax(probs,
                             dim=1)
        return preds

### Training and Testing the model

We create an instance of the hindi model
Define the loss function : <code>Entropy Loss</code>
<br>
Define the grad fucntion : <code>Stochastic Gradient Descent</code>

In [227]:
# Creating an instance of the hindi model
hindi_model = DigitModel()

# Creating the loss function 
loss_fn = nn.CrossEntropyLoss()

# Creating the grad fucntion
optimiser = torch.optim.SGD(
    hindi_model.parameters(),
    lr=0.001
)

# Epoch Count 
EPOCHS =int(1e3)
torch.manual_seed(42)

In [228]:
# Tesnoring the data
X_train = torch.tensor(X_train).type(torch.float)
X_test = torch.tensor(X_test).type(torch.float)
y_train = torch.tensor(y_train).type(torch.float)
y_test = torch.tensor(y_test).type(torch.float)


In [229]:
foo_list = [0]*10
foo_list[int(y_train[0])] = 1
foo_list = torch.tensor(foo_list)

inp = hindi_model(X_train[0])
inp = torch.softmax(inp,dim=0)


loss_fn(inp,y_train[0].type(torch.long))

tensor(2.2741, grad_fn=<NllLossBackward0>)

In [230]:
torch.manual_seed(42)
# training and testing loop
for epoch in range(EPOCHS):
    # Training part
    hindi_model.train()
    
    y_logits = hindi_model(X_train)
    y_probs = torch.softmax(y_logits,dim=1)
    y_preds = hindi_model.logits_to_preds(y_logits)
    
    '''foo_train = []    
    for i in y_train:
        foo_list = [0]*10
        foo_list[int(i)] = 1
        foo_train.append(foo_list)
    foo_train = torch.tensor(foo_train).type(torch.float)'''
    
    loss = loss_fn(
        y_logits,
        y_train.type(torch.long)
    )
    
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()
    
    # Testing code
    hindi_model.eval()
    
    with torch.inference_mode():
        y_logits = hindi_model(X_test)
        y_probs = torch.softmax(y_logits,dim=1)
        y_preds = hindi_model.logits_to_preds(y_logits)
        '''foo_test = []    
        for i in y_test:
            foo_list = [0]*10
            foo_list[int(i)] = 1
            foo_test.append(foo_list)
        foo_test = torch.tensor(foo_test).type(torch.float)'''
        test_loss = loss_fn(y_logits, y_test.type(torch.long))
        
        
    if epoch%10 == 0:
        print(f"Epoch: {epoch:.2f}, Loss: {loss:.2f}, test_loss: {test_loss:.2f}")
        
        


Epoch: 0.00, Loss: 2.32, test_loss: 2.26
Epoch: 10.00, Loss: 2.32, test_loss: 2.26
Epoch: 20.00, Loss: 2.32, test_loss: 2.26
Epoch: 30.00, Loss: 2.32, test_loss: 2.26
Epoch: 40.00, Loss: 2.32, test_loss: 2.26
Epoch: 50.00, Loss: 2.32, test_loss: 2.26


Epoch: 60.00, Loss: 2.32, test_loss: 2.26
Epoch: 70.00, Loss: 2.32, test_loss: 2.26
Epoch: 80.00, Loss: 2.32, test_loss: 2.26
Epoch: 90.00, Loss: 2.32, test_loss: 2.26
Epoch: 100.00, Loss: 2.32, test_loss: 2.26
Epoch: 110.00, Loss: 2.32, test_loss: 2.26
Epoch: 120.00, Loss: 2.32, test_loss: 2.26
Epoch: 130.00, Loss: 2.32, test_loss: 2.26
Epoch: 140.00, Loss: 2.32, test_loss: 2.26
Epoch: 150.00, Loss: 2.32, test_loss: 2.26
Epoch: 160.00, Loss: 2.31, test_loss: 2.26
Epoch: 170.00, Loss: 2.31, test_loss: 2.26
Epoch: 180.00, Loss: 2.31, test_loss: 2.26
Epoch: 190.00, Loss: 2.31, test_loss: 2.26
Epoch: 200.00, Loss: 2.31, test_loss: 2.26
Epoch: 210.00, Loss: 2.31, test_loss: 2.26
Epoch: 220.00, Loss: 2.31, test_loss: 2.26
Epoch: 230.00, Loss: 2.31, test_loss: 2.26
Epoch: 240.00, Loss: 2.31, test_loss: 2.26
Epoch: 250.00, Loss: 2.31, test_loss: 2.26
Epoch: 260.00, Loss: 2.31, test_loss: 2.26
Epoch: 270.00, Loss: 2.31, test_loss: 2.26
Epoch: 280.00, Loss: 2.31, test_loss: 2.26
Epoch: 290.00, 

In [231]:
hindi_model.eval()
with torch.inference_mode():
    test = hindi_model(X_test)
    test_preds = hindi_model.logits_to_preds(test)
    
from torchmetrics import Accuracy

# Setup Metric
torchmetric_accuracy = Accuracy(task="multiclass",num_classes=10)

# Calculate accuracy
torchmetric_accuracy(test_preds, y_test)
    


tensor(0.0943)